In [ ]:
import os
import numpy as np
import pickle
import imageio
import pandas as pd
import keras
import keras.losses
import keras.backend as K
from keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import seaborn as sn

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
plt.rcParams['figure.figsize'] = [13, 4]

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

In [ ]:
def matthews_correlation(y_true, y_pred):
    '''Calculates the Matthews correlation coefficient measure for quality
    of binary classification problems.
    '''
    '''  
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)

    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())
    '''
    return mcc(y_true, y_pred)

In [ ]:
def plot_history(history, filename):
    loss_list = [s for s in history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.keys() if 'acc' in s and 'val' in s]
    f1_list = [s for s in history.keys() if 'f1' in s and 'val' not in s]
    val_f1_list = [s for s in history.keys() if 'f1' in s and 'val' in s]
    prec_list = [s for s in history.keys() if 'precision' in s and 'val' not in s]
    val_prec_list = [s for s in history.keys() if 'precision' in s and 'val' in s]
    rec_list = [s for s in history.keys() if 'recall' in s and 'val' not in s]
    val_rec_list = [s for s in history.keys() if 'recall' in s and 'val' in s]

    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history[loss_list[0]]) + 1)
    
    ## Loss
    #plt.figure(1)
    plt.subplot(5, 2, 1)
    for l in loss_list:
        plt.plot(epochs, history[l], 'b', label='Training loss (' + str(str(format(history[l][-1],'.5f'))+')'))
    
    plt.title('Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Training Loss')
    plt.legend()
    
    plt.subplot(5,2,2)
    for l in val_loss_list:
        plt.plot(epochs, history[l], 'g', label='Validation loss (' + str(str(format(history[l][-1],'.5f'))+')'))
    
    plt.title('Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Validation Loss')
    plt.legend()
    #plt.savefig(filename)
    plt.ylim(0, 1)

    ## Accuracy
    if(len(acc_list) != 0):
        #plt.figure(2)
        plt.subplot(5, 2, 3)
        for l in acc_list:
            plt.plot(epochs, history[l], 'b', label='Training accuracy (' + str(format(history[l][-1],'.5f'))+')')
            
        plt.title('Training Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Training Accuracy')
        plt.legend()
        plt.ylim(0,1)
        #plt.savefig(filename)

        plt.subplot(5 ,2, 4)
        for l in val_acc_list:    
            plt.plot(epochs, history[l], 'g', label='Validation accuracy (' + str(format(history[l][-1],'.5f'))+')')

        plt.title('Validation Accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Validation Accuracy')
        plt.legend()
        plt.ylim(0,1)
        #plt.savefig(filename)

    ## F1
    if(len(f1_list) != 0):
        #plt.figure(2)
        plt.subplot(5, 2, 5)
        for l in f1_list:
            plt.plot(epochs, history[l], 'b', label='Training F1 (' + str(format(history[l][-1],'.5f'))+')')
        plt.title('Training F1 Score')
        plt.xlabel('Epochs')
        plt.ylabel('Training F1 Score')
        plt.legend()
        #plt.savefig(filename)  
        
        plt.subplot(5, 2, 6)
        for l in val_f1_list:    
            plt.plot(epochs, history[l], 'g', label='Validation F1 (' + str(format(history[l][-1],'.5f'))+')')

        plt.title('Validation F1 Score')
        plt.xlabel('Epochs')
        plt.ylabel('Validation F1 Score')
        plt.legend()
        #plt.savefig(filename)
        
    ## Precision
    if(len(prec_list) != 0):
        #plt.figure(2)
        plt.subplot(5, 2, 7)
        for l in prec_list:
            plt.plot(epochs, history[l], 'b', label='Training Precision (' + str(format(history[l][-1],'.5f'))+')')
        plt.title('Training Precision')
        plt.xlabel('Epochs')
        plt.ylabel('Training Precision')
        plt.legend()
        #plt.savefig(filename)
        
        plt.subplot(5, 2, 8)
        for l in val_prec_list:    
            plt.plot(epochs, history[l], 'g', label='Validation Precision (' + str(format(history[l][-1],'.5f'))+')')

        plt.title('Validation Precision')
        plt.xlabel('Epochs')
        plt.ylabel('Validation Precision')
        plt.legend()
        #plt.savefig(filename)

    ## Recall
    if(len(rec_list) != 0):
        #plt.figure(2)
        plt.subplot(5, 2, 9)
        for l in rec_list:
            plt.plot(epochs, history[l], 'b', label='Training Recall (' + str(format(history[l][-1],'.5f'))+')')
        plt.title('Training Recall')
        plt.xlabel('Epochs')
        plt.ylabel('Training Recall')
        plt.legend()
        #plt.savefig(filename)

        
        plt.subplot(5, 2, 10)
        for l in val_rec_list:    
            plt.plot(epochs, history[l], 'g', label='Validation Recall (' + str(format(history[l][-1],'.5f'))+')')

        plt.title('Validation Recall')
        plt.xlabel('Epochs')
        plt.ylabel('Validation Recall')
        plt.legend()
        #plt.savefig(filename)

    plt.tight_layout()
    plt.show()

In [ ]:
dir = "./Models/JSON/tests"
fileName = "noKFold3Class_lr-5.json"
f = plt.figure(figsize=(13,13))
history = json.load(open(os.path.join(dir, fileName) ,"r"))
saveTo = os.path.join("./Graphs", (fileName[:-5]+".png"))
plot_history(history, saveTo)

In [ ]:
model = load_model("./Models/model/tests/noKFold3Class_lr-5.h5", 
                   custom_objects={"precision": precision_m, 
                                   "recall_m": recall_m}) 
plot_model(model, to_file='./Graphs/model_vis.png')

In [ ]:
model = load_model("./Models/model/tests/noKFold3Class_lr-5.h5", 
                   custom_objects={"f1_m": f1_m,
                                   "precision_m": precision_m, 
                                   "recall_m": recall_m}) 
                   #, custom_objects={'matthews_correlation': matthews_correlation})
X = []
y = []
noPlaqueErrors = []
noPlaqueErrorsPred = []
calErrors = []
calErrorsPred = []
fibErrors = []
fibErrorsPred = []
errors = 0

dir = "./Data/Patches/NoPlaque"
for filename in os.listdir(dir):
    img = np.array(imageio.imread(os.path.join(dir, filename)))[:,:,0:1]
    #img = np.load(os.path.join(dir, filename))[:,:,0]
    X.append(img)
    y.append([1,0,0])
    val = np.expand_dims(img, axis=0)
    if np.argmax(model.predict(val)) != 0:
        errors += 1
        noPlaqueErrors.append(img)
        noPlaqueErrorsPred.append(model.predict(val))

#X = np.array(X)
#y = np.array(y)
#for i in range(20):
#    print(y[i])
#    val = np.expand_dims(X[i][:,:,0:1], axis=0)
#    print(np.argmax(model.predict(val)))
#    plt.imshow(X[i]/256)
#    plt.show()

dir = "./Data/Patches/Plaque/Aug/cal"
for filename in os.listdir(dir):
    img = np.array(imageio.imread(os.path.join(dir, filename)))[:,:,0:1]
    #img = np.load(os.path.join(dir, filename))[:,:,0]
    X.append(img)
    y.append([0,1,0])
    val = np.expand_dims(img, axis=0)
    if np.argmax(model.predict(val)) != 1:
        errors += 1
        calErrors.append(img)
        calErrorsPred.append(model.predict(val))
    
#X = np.array(X)
#y = np.array(y)
#for i in range(20):
#    print(y[i])
#    val = np.expand_dims(X[i][:,:,0:1], axis=0)
#    print(np.argmax(model.predict(val)))
#    plt.imshow(X[i]/256)
#    plt.show()

dir = "./Data/Patches/Plaque/Aug/fibrous"
for filename in os.listdir(dir):
    img = np.array(imageio.imread(os.path.join(dir, filename)))[:,:,0:1]
    #img = np.load(os.path.join(dir, filename))[:,:,0]
    X.append(img)
    y.append([0,0,1])
    val = np.expand_dims(img, axis=0)
    if np.argmax(model.predict(val)) != 2:
        errors += 1
        fibErrors.append(img)
        fibErrorsPred.append(model.predict(val))
#X = np.array(X)
#y = np.array(y)
#for i in range(20):
#    print(y[i])
#    val = np.expand_dims(X[i][:,:,0:1], axis=0)
#    print(np.argmax(model.predict(val)))
#    plt.imshow(X[i]/256)
#    plt.show()

X = np.array(X)
y = np.array(y)


In [ ]:
plt.imshow(noPlaqueErrors[5][:,:,0])
plt.show()

In [ ]:
dir = "./Data/Patches/NoPlaque"
filename = os.listdir(dir)[0]
img = np.array(imageio.imread(os.path.join(dir, filename)))[:,:,0]
plt.imshow(img[:,:])
plt.show()

In [ ]:
errors

In [ ]:
(errors/len(X))*100

In [ ]:
y_pred = model.predict(X)
cmatrix = np.zeros((3,3))
for i,y_our in enumerate(y_pred):
    cmatrix[np.argmax(y_our)][np.argmax(y[i])] += 1

In [ ]:
df_cm = pd.DataFrame(cmatrix, index = ["Normal", "Cal", "Fib"], columns = ["Normal", "Cal", "Fib"])
plt.figure(figsize = (10,7))
snheatmap = sn.heatmap(df_cm, annot=True,  fmt='g', cmap="Blues")
snheatmap.get_figure().savefig("Graphs/CONFUSION_MATRIX_1_KFOLD_TEST.png")

In [ ]:
df_cm = pd.DataFrame(cmatrix[1:,1:], index = ["Cal", "Fib"], columns = ["Cal", "Fib"])
plt.figure(figsize = (10,7))
snheatmap = sn.heatmap(df_cm, annot=True,  fmt='g', cmap="Blues")
snheatmap.get_figure().savefig("Graphs/CONFUSION_MATRIX_2_TEST.png")

print("Acc plaque detection: ", 1-(cmatrix[1:,1:][0][1] + cmatrix[1:,1:][1][0])/np.sum(cmatrix[1:,1:]))

In [ ]:
cmatrix = np.zeros((2,2))
for i,y_our in enumerate(y_pred):
    val = min(1, np.argmax(y_our))
    val1 = min(1, np.argmax(y[i]))
    cmatrix[val][val1] += 1
df_cm = pd.DataFrame(cmatrix, index = ["Normal", "Plaque"], columns = ["Normal", "Plaque"])
plt.figure(figsize = (10,7))
snheatmap = sn.heatmap(df_cm, annot=True,  fmt='g', cmap="Blues")
snheatmap.get_figure().savefig("Graphs/CONFUSION_MATRIX_3_KFOLD_TEST_2CLASS.png")

print("Acc plaque detection: ", 1-(cmatrix[0][1] + cmatrix[1][0])/np.sum(cmatrix))